In [8]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc import psd as psd_func
from pycbc.filter import sigma
from pycbc.noise import frequency_noise_from_psd
from simple_pe.waveforms import make_waveform, calculate_mode_snr

import logging
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# Generate single eccentric waveform as 'data'

In [2]:
# Generate psd
ifos = ['H1']
tlen = 32
sample_rate = 4096
f_low = 20
psds = {'H1': 'aLIGOZeroDetHighPower',
        'f_low': f_low,
        'f_high': int(sample_rate/2),
        'length': tlen,
        'delta_f': 1. / tlen
        }
psd = {}
for ifo in ifos:
    psd[ifo] = psd_func.analytical.from_string(psds[ifo], psds['length'] * psds['f_high'] + 1, psds['delta_f'],
                                                     psds['f_low'])

# Generate TEOBResumS-Dali waveform
data_pars = {'ecc10sqrd': 0.014**2, 'chirp_mass': 32, 'symmetric_mass_ratio': 0.249, 'chi_eff': 0, 'distance': 3000}
df = 1/tlen
flen = (tlen*sample_rate)//2 + 1
raw_data = make_waveform(data_pars, df, f_low, flen, approximant='TEOBResumS-Dali')

sig = sigma(raw_data, psd['H1'], low_frequency_cutoff=f_low, high_frequency_cutoff=psd['H1'].sample_frequencies[-1])
print(f'sigma: {sig}')

# Add gaussian noise
# data = {'H1': (raw_data + frequency_noise_from_psd(psd['H1'], seed=12345)).to_timeseries()}
data = {'H1': (raw_data).to_timeseries()}

/home/ben.patterson/.conda/envs/igwn_eccentric/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sigma: 14.395556742681567


# Generate eccentric harmonics and match filter data

In [3]:
# Generate TEOBResumS-Dali waveform
harm_pars = {'ecc10sqrd': 0.014**2, 'chirp_mass': 32, 'symmetric_mass_ratio': 0.249, 'chi_eff': 0, 'distance': 1}
harm_dict = make_waveform(harm_pars, df, f_low, flen, approximant='TEOBResumS-Dali-Harms', n_ecc_harms=4, n_ecc_gen=6)

# Match filter harmonics
harm_perp = {}
for key in harm_dict.keys():
    harm_perp[key] = harm_dict[key] / sigma(harm_dict[key], psd[ifos[0]], low_frequency_cutoff=f_low,
                                            high_frequency_cutoff=psd[ifos[0]].sample_frequencies[-1])
mode_SNRs, _ = calculate_mode_snr(data[ifos[0]], psd[ifos[0]], harm_perp, data[ifos[0]].sample_times[0],
                                  data[ifos[0]].sample_times[-1], f_low, harm_dict.keys(), dominant_mode=0)
for key in mode_SNRs.keys():
    print(f'rho_{key}: {np.abs(mode_SNRs[key]):.2f}, phi_{key}: {np.angle(mode_SNRs[key]):.2f}')

/home/ben.patterson/.conda/envs/igwn_eccentric/lib/python3.10/site-packages/pycbc/types/array.py:217: RuntimeWarning: divide by zero encountered in divide
  ret = getattr(ufunc, method)(*inputs, **kwargs)


rho_0: 14.30, phi_0: -1.00
rho_1: 0.17, phi_1: 2.16
rho_-1: 0.51, phi_-1: 3.05
rho_2: 0.24, phi_2: 0.35


In [58]:
# Generate TEOBResumS-Dali waveform
harm_pars = {'ecc10sqrd': 0.014**2, 'chirp_mass': 32, 'symmetric_mass_ratio': 0.249, 'chi_eff': 0, 'distance': 1}
harm_dict = make_waveform(harm_pars, df, f_low, flen, approximant='TEOBResumS-Dali-Harms', n_ecc_harms=4, n_ecc_gen=6)

# Match filter harmonics
harm_perp = {}
for key in harm_dict.keys():
    harm_perp[key] = harm_dict[key] / sigma(harm_dict[key], psd[ifos[0]], low_frequency_cutoff=f_low,
                                            high_frequency_cutoff=psd[ifos[0]].sample_frequencies[-1])
mode_SNRs, _ = calculate_mode_snr(data[ifos[0]], psd[ifos[0]], harm_perp, data[ifos[0]].sample_times[0],
                                  data[ifos[0]].sample_times[-1], f_low, harm_dict.keys(), dominant_mode=0)
for key in mode_SNRs.keys():
    print(f'rho_{key}: {np.abs(mode_SNRs[key]):.2f}, phi_{key}: {np.angle(mode_SNRs[key]):.2f}')

rho_0: 14.35, phi_0: -1.00
rho_1: 0.29, phi_1: 1.70
rho_-1: 0.33, phi_-1: 2.85
rho_2: 0.02, phi_2: -0.99
